# Домашнее задание 2
## Named Entity Recognition and Event Extraction

deadline: 06 ноября 2022, 23:59

В этом домашнем задании вы будете работать с двумя корпусами - LitBank и MultiNERD_EN.
Первый корпус (LitBank) собран из популярных художественных произведений на английском языке и содержит разметку по именованным сущностям и событиям, корпус состоит из 100 текстов по примерно 2000 слов каждый. 

Корпус описан в статьях:
* David Bamman, Sejal Popat, Sheng Shen, An Annotated Dataset of Literary Entities http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/naacl2019_literary_entities.pdf
* Matthew Sims, Jong Ho Park, David Bamman, Literary Event Detection,  http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/acl2019_literary_events.pdf

Корпус доступен в репозитории проекта:  https://github.com/dbamman/litbank

Статья и код, использованный для извлечения именованных сущностей: 
* Meizhi Ju, Makoto Miwa and Sophia Ananiadou, A Neural Layered Model for Nested Named Entity Recognition, https://github.com/meizhiju/layered-bilstm-crf

Структура корпуса устроена так. 
Первый уровень: 
* entities -- разметка по сущностям
* events -- разметка по сущностям


В корпусе используются 6 типов именованных сущностей: PER, LOC, ORG, FAC, GPE, VEH (имена, локации, организации, помещения, топонимы, средства перемещения), допускаются вложенные сущности. 

События выражается одним словом - *триггером*, которое может быть глаголом, прилагательным и существительным. В корпусе описаны события, которые действительно происходят и не имеют гипотетического характера. 
Пример: she *walked* rapidly and resolutely, здесь *walked* -- триггер события. Типы событий не заданы. 


Второй уровень:
* brat -- рабочие файлы инструмента разметки brat, ann-файлы содержат разметку, txt-файлы – сырые тексты 
* tsv -- tsv-файлы содержат разметку в IOB формате,

Второй корпус (MultiNERD_EN) состоит из 164 тысяч предложений по различным тематикам. От предыдущего корпуса он отличается большим числом (15) типов именованных сущностей. Также одной из сущностей является сущность-событие (EVE). Каждая сущность может состоять как из одного, так и нескольких слов.

Корпус доступен в репозитории проекта:  https://github.com/Babelscape/multinerd

В статьях и репозиториях вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и использовать повторно. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 4-х человек.
2. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо в ipython-тетрадке. При этом прикладывать ipynb-файл обязательно. 
43. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
4. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
5. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
6. Плагиат и любое недобросовестное цитирование приводит к обнулению оценки. 


## Часть 1. [2 балла] Эксплоративный анализ 
1. [0.4 балла] Найдите топ 10 (по частоте) именованных сущностей каждого из типов для каждого корпуса. Произведите сравнение частотности по совпадающим сущностям при сопоставлении корпусов друг с другом.
2. [0.4 балла] Найдите топ 10 (по частоте) частотных триггеров событий (на основе LitBank). Произведите анализ из пункта 1 для именованной сущности типа EVE (на основе MultiNERD_EN). 
3. [0.6 балла] Кластеризуйте все уникальные триггеры событий (LitBank), используя эмбеддинги слов и любой алгоритм кластеризации (например, аггломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий? 
4. [0.6 балла] Отдельно произведите анализ из п. 3 для сущностей типа EVE (на основе MultiNERD_EN). Сравните результаты из п. 3 и п. 4.

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу LitBank и сравните кластеры триггеров и выделенные темы: есть ли схожие паттерны в тематической модели и в структуре кластеров?

В следующих частях домашнего задания вам понадобится train-test-dev разбиения. Авторы статей предлагают следующую структуру разбиения для корпуса LitBank: обучающее множество – 80 книг, валидационное – 10 книг, тестовое – 10 книг. Предложения из одного источника не должны попадать в разные сегменты разбиения.
Для корпуса MultiNERD_EN произведите стратифицированное разбиение по предложениям в соотношении 80%:10%:10%. Стратификацию производить в отношении пропорции по именованным сущностям (понятно, что с учётом всех вводных идеального разбиения не получится, но старайтесь сохранить распределения по типам сущностей для каждой и подвыборок).


## Часть 2. [3 балла] Извлечение именованных сущностей
Для каждой корпуса необходимо осуществить раздельно следующую процедуру:
1. [0.75 балла] Обучите стандартную модель для извлечения именованных сущностей, CNN-BiLSTM-CRF, для извлечения именованных *низкоуровневых именованных сущностей*, т.е. для самых коротких из вложенных сущностей. 
Модель устроена так: сверточная сеть на символах + эмбеддинги слов + двунаправленная LSTM сеть (модель последовательности) + CRF (глобальная нормализация).
2. [0.75 балла] Замените часть модели на символах и словах (CNN + эмбеддинги словах) на ELMo и / или BERT. Должна получиться модель ELMo / BERT + BiLSTM + CRF. 
3. [1.5 балла] Замените модель последовательности (BiLSTM) на другой слой, например, на Transformer. Должна получиться модель CNN  + Transformer + CRF.
В результате у вас будет 2 набора моделей - проверьте качество извлечения на тестовых данных.
- Примените модели первого корпуса ко всем (!!!) предложениям второго корпуса и наоборот.
- Составьте отчёт по качеству работы моделей в терминах извлечения по типам сущностей, которые встречаются в обоих корпусах. Метрику выберите самостоятельно.
- Проанализируйте ошибки моделей в терминах предсказания сущностям первого корпуса специфических типов для второго корпуса. И наоборот - когда специфическим типам второго корпуса модель, обученная на LitBank, прсваивает одну из своих категорий.

[бонус] Дообучите BERT для извлечения именованных сущностей.

[бонус] Используйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]

[бонус] Модифицируйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]: вместо эмбеддингов слов используйте ELMo и/или BERT. 

## Часть 3. [2 балла] Извлечение событий 

1. [0.75 балла] Используйте BiLSTM на эмбеддингах слов для извлечения триггеров событий (на основе LitBank). 

2. [0.75 балла] Замените часть модели на словах на ELMo и/или BERT. Должна получиться модель ELMo / BERT + BiLSTM.

3. [0.5 балла] Проверьте "извлекающую" силу модели на данных MultiNERD_EN для событий типа EVE. 

[бонус] Предобучите BiLSTM как языковую модель. Дообучите ее для извлечения триггеров. 

[бонус] Дообучите BERT для извлечения триггеров событий. 

## Часть 4. [2 балла] Одновременное извлечение именованных сущностей и событий 
1. [0.75 балла] Обучите модель для совместного извлечения именованных сущностей и триггеров событий только на основе LitBank (!). У модели должен быть общий энкодер (например, CNN + BiLSMT, ELMo + BiLSTM, BERT + BiLSTM) и два декодера: один отвечает за извлечение именованных сущностей, другой отвечает за извлечение триггеров событий.

2. [0.75 балла] Создайте единую обучающую подвыборку, единую валидационную и единую тестовую на основе разбиений, которые вы произвели в первой части и обучите модель. (Единую - то есть на основе обоих корпусов сразу)

3. [0.5 балла] Сравните предсказательную силу модели из п.1 и модели из п.2 как на совместных подвыборках, так и на раздельных. Проанализируйте полученный результат. Приводит ли обогащение дополнительными данными к улучшению "извлекающей" способности модели?

[бонус] Добавьте в модель механизм внимания, таким способом, который покажется вам разумным.

[бонус] Визуализируйте карты механизма внимания (attention). 

## Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?

<span style="color:pink">Авторы работы: Токкожин Арсен, Соколов Ян, Екимов Егор, Гвасалия Лукас</span>
# Решение:

## Часть 1. [2 балла] Эксплоративный анализ 
1. [0.4 балла] Найдите топ 10 (по частоте) именованных сущностей каждого из типов для каждого корпуса. Произведите сравнение частотности по совпадающим сущностям при сопоставлении корпусов друг с другом.
2. [0.4 балла] Найдите топ 10 (по частоте) частотных триггеров событий (на основе LitBank). Произведите анализ из пункта 1 для именованной сущности типа EVE (на основе MultiNERD_EN). 
3. [0.6 балла] Кластеризуйте все уникальные триггеры событий (LitBank), используя эмбеддинги слов и любой алгоритм кластеризации (например, аггломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий? 
4. [0.6 балла] Отдельно произведите анализ из п. 3 для сущностей типа EVE (на основе MultiNERD_EN). Сравните результаты из п. 3 и п. 4.

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу LitBank и сравните кластеры триггеров и выделенные темы: есть ли схожие паттерны в тематической модели и в структуре кластеров?

In [1]:
import os
from brat_parser import get_entities_relations_attributes_groups
from collections import Counter

entities_folder_path = "litbank_entities/brat"
events_folder_path = "litbank_events/brat"
multinerd_path = "multinerd_en.tsv"

#### Топ 10 именованных сущностей для корпуса Litbank

In [2]:
entities = ["PER", "FAC", "LOC", "ORG", "GPE", "VEH"]

In [3]:
def count_entities(path_to_folder, entities_names):
    result = {}
    for entity in entities_names:
        result[entity] = Counter()

    for filename in os.listdir(path_to_folder):
        if filename.endswith(".ann"):
            entities, relations, attributes, groups = get_entities_relations_attributes_groups(os.path.join(path_to_folder, filename))
            for item in entities:
                result[entities[item].type][entities[item].text] += 1 

    return result

In [4]:
entities_ans = count_entities(entities_folder_path, entities)
for entity in entities_ans:
    print("-------------------------------------")
    print(entity)
    temp = entities_ans[entity].most_common()[:10]
    for item in temp:
        print(item)

-------------------------------------
PER
('Mr.', 148)
('Miss', 133)
('Mrs.', 132)
('sir', 50)
('Sir', 45)
('men', 40)
('my mother', 40)
('Cameron', 38)
('his wife', 37)
('Mr', 37)
-------------------------------------
FAC
('home', 65)
('the house', 52)
('here', 39)
('there', 39)
('the room', 34)
('the garden', 23)
('the street', 14)
('the hall', 13)
('the road', 13)
('the place', 12)
-------------------------------------
LOC
('the world', 72)
('the sea', 27)
('the river', 22)
('the country', 20)
('there', 18)
('the earth', 16)
('sea', 16)
('the valley', 13)
('this world', 12)
('the woods', 9)
-------------------------------------
ORG
('the army', 7)
('the Church', 4)
('the Committee of Public Safety', 4)
('the Colonial Office', 4)
('college', 3)
('Harvard', 3)
('Carston , Waite and Co.', 2)
('the hospital', 2)
('the C.C.H.', 2)
("the Bank of Leichardt 's Land", 2)
-------------------------------------
GPE
('London', 40)
('England', 32)
('there', 21)
('the town', 21)
('New York', 16)
(

#### Топ 10 именованных сущностей для корпуса MultiNERD_EN

In [5]:
import pandas as pd
import csv
col_names = ["Index", "Token", "Label", "BabelNet ID", "Wikidata ID", "Wikipedia ID", "Wikipedia Title", "Definition", "Image URL", "Smth"]
multinerd_data = pd.read_csv(multinerd_path, sep="\t", names=col_names, index_col=False , quoting=csv.QUOTE_NONE)

In [6]:
multinerd_data.head()

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL,Smth
0,0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,by,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,James,B-PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...,NaN
3,3,Cameron,I-PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,and,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print(multinerd_data["Smth"].size)
print(multinerd_data["Smth"].isna().count())

3568155
3568155


видим что последняя колонка полнстью состоит из пропусков, так что ее можно удалить

In [8]:
multinerd_data=multinerd_data.drop(["Smth"], axis=1)

In [9]:
multinerd_data.head()

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL
0,0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN
1,1,by,O,NaN,NaN,NaN,NaN,NaN,NaN
2,2,James,B-PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...
3,3,Cameron,I-PER,NaN,NaN,NaN,NaN,NaN,NaN
4,4,and,O,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
multinerd_data.loc[multinerd_data['Label'] != 'O', 'Label'] = multinerd_data.loc[multinerd_data['Label'] != 'O', 'Label'].str[2:]

In [11]:
multinerd_data.head(10)

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL
0,0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN
1,1,by,O,NaN,NaN,NaN,NaN,NaN,NaN
2,2,James,PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...
3,3,Cameron,PER,NaN,NaN,NaN,NaN,NaN,NaN
4,4,and,O,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Charles,PER,bn:02320763n,Q3666404,2956611.0,Charles_H._Eglee,Charles Hamilton Eglee is an American film and...,https://upload.wikimedia.org/wikipedia/commons...
6,6,H.,PER,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Eglee,PER,NaN,NaN,NaN,NaN,NaN,NaN
8,8,",",O,NaN,NaN,NaN,NaN,NaN,NaN
9,9,it,O,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
all_labels = list(multinerd_data["Label"])
cnt = Counter()
for label in all_labels:
    cnt[label] += 1
cnt

Counter({'O': 3111887,
         'PER': 155669,
         'LOC': 112519,
         'MEDIA': 19673,
         'TIME': 5584,
         'EVE': 8525,
         'ORG': 78179,
         'PLANT': 12674,
         'ANIM': 22284,
         'FOOD': 15006,
         'DIS': 18713,
         'CEL': 4341,
         'SUPER': 774,
         'VEHI': 1114,
         'INST': 822,
         'PHY': 190,
         'BIO': 201})

In [13]:
multinerd_entities_names = cnt.keys()

In [14]:
df = multinerd_data.groupby(['Label', 'Token'])[['Index']].count().reset_index()

In [15]:
df = df.rename(columns={
    "Index":"Amount",
})

In [16]:
multinerd_entities = {}
for entity in multinerd_entities_names:
    print("-------------------------------------")
    print(entity)
    temp = df[df["Label"] == entity].sort_values(["Amount"], ascending=False)
    print(temp[["Token","Amount"]].head(10).to_string(index=False))
    multinerd_entities[entity] = list(temp["Token"].head(10))

-------------------------------------
O
Token  Amount
    ,  194114
  the  169553
    .  164196
   of   94883
  and   90076
   in   83052
    "   72596
    a   65926
   to   58243
   is   40965
-------------------------------------
PER
  Token  Amount
   John    2097
 George    1114
  David    1026
William     911
  James     879
Michael     824
 Robert     794
   Paul     745
Charles     691
Richard     660
-------------------------------------
LOC
    Token  Amount
   United    2357
      New    2030
   States    1604
    South    1568
Australia    1103
  America    1035
       of     999
    North     998
   Africa     943
    River     919
-------------------------------------
MEDIA
   Token  Amount
Tomatoes    1466
  Rotten    1466
     The     954
      of     498
     the     452
      's     168
       :     131
     You     119
     and     108
    Show     107
-------------------------------------
TIME
      Token  Amount
        Day     222
        Age     214
     Middle   

In [17]:
multinerd_entities

{'O': [',', 'the', '.', 'of', 'and', 'in', '"', 'a', 'to', 'is'],
 'PER': ['John',
  'George',
  'David',
  'William',
  'James',
  'Michael',
  'Robert',
  'Paul',
  'Charles',
  'Richard'],
 'LOC': ['United',
  'New',
  'States',
  'South',
  'Australia',
  'America',
  'of',
  'North',
  'Africa',
  'River'],
 'MEDIA': ['Tomatoes',
  'Rotten',
  'The',
  'of',
  'the',
  "'s",
  ':',
  'You',
  'and',
  'Show'],
 'TIME': ['Day',
  'Age',
  'Middle',
  'Ages',
  'Great',
  'Depression',
  'Renaissance',
  'of',
  'Christmas',
  'Bronze'],
 'EVE': ['War',
  'World',
  'II',
  'Civil',
  'Cup',
  'American',
  'Revolution',
  'Festival',
  'League',
  'Film'],
 'ORG': ['University',
  'of',
  'United',
  'Party',
  'City',
  'New',
  'FC',
  'National',
  'Town',
  'Union'],
 'PLANT': ['plant',
  'rice',
  'oak',
  'wheat',
  'pepper',
  'shrub',
  'deciduous',
  '"',
  'tree',
  'pine'],
 'ANIM': ['moth',
  'sea',
  'snail',
  'fish',
  'deer',
  'trout',
  'Erebidae',
  'Noctuidae',


In [18]:
litbank_entities = {}
for entity in entities_ans:
    temp = entities_ans[entity].most_common()[:10]
    litbank_entities[entity] = list()
    for item in temp:
        litbank_entities[entity].append(item[0])

In [19]:
litbank_entities

{'PER': ['Mr.',
  'Miss',
  'Mrs.',
  'sir',
  'Sir',
  'men',
  'my mother',
  'Cameron',
  'his wife',
  'Mr'],
 'FAC': ['home',
  'the house',
  'here',
  'there',
  'the room',
  'the garden',
  'the street',
  'the hall',
  'the road',
  'the place'],
 'LOC': ['the world',
  'the sea',
  'the river',
  'the country',
  'there',
  'the earth',
  'sea',
  'the valley',
  'this world',
  'the woods'],
 'ORG': ['the army',
  'the Church',
  'the Committee of Public Safety',
  'the Colonial Office',
  'college',
  'Harvard',
  'Carston , Waite and Co.',
  'the hospital',
  'the C.C.H.',
  "the Bank of Leichardt 's Land"],
 'GPE': ['London',
  'England',
  'there',
  'the town',
  'New York',
  'town',
  'France',
  'Europe',
  'the country',
  'Rome'],
 'VEH': ['the ship',
  'the car',
  'the train',
  'the boat',
  'boats',
  'the carriage',
  'ships',
  'a carriage',
  'the waggon',
  'the coach']}

In [20]:
for entity in list(set(litbank_entities) & set(multinerd_entities)):
    print("----------------------------")
    print(entity)
    print("Litbank:")
    print(litbank_entities[entity])
    print("Multinerd:")
    print(multinerd_entities[entity])

----------------------------
ORG
Litbank:
['the army', 'the Church', 'the Committee of Public Safety', 'the Colonial Office', 'college', 'Harvard', 'Carston , Waite and Co.', 'the hospital', 'the C.C.H.', "the Bank of Leichardt 's Land"]
Multinerd:
['University', 'of', 'United', 'Party', 'City', 'New', 'FC', 'National', 'Town', 'Union']
----------------------------
PER
Litbank:
['Mr.', 'Miss', 'Mrs.', 'sir', 'Sir', 'men', 'my mother', 'Cameron', 'his wife', 'Mr']
Multinerd:
['John', 'George', 'David', 'William', 'James', 'Michael', 'Robert', 'Paul', 'Charles', 'Richard']
----------------------------
LOC
Litbank:
['the world', 'the sea', 'the river', 'the country', 'there', 'the earth', 'sea', 'the valley', 'this world', 'the woods']
Multinerd:
['United', 'New', 'States', 'South', 'Australia', 'America', 'of', 'North', 'Africa', 'River']


Сравнение: одинаковые сущности в разных датасетах трактуются по-разному. Видим что в датасете multinerd большую популярность среди сущностей персон(PER) и локаций(LOC) имеют имена собственные, нежели в датасете litbank.

#### Топ 10 частотных триггеров событий для корпуса Litbank

In [21]:
def count_events(path_to_folder):
    cnt = Counter()

    for filename in os.listdir(path_to_folder):
        if filename.endswith(".ann"):
            entities, relations, attributes, groups = get_entities_relations_attributes_groups(os.path.join(path_to_folder, filename))
            for item in entities:
                cnt[entities[item].text] += 1 

    return cnt

In [22]:
litbank_events = count_events(events_folder_path).most_common()[:10]
litbank_events

[('said', 464),
 ('came', 95),
 ('looked', 92),
 ('went', 92),
 ('asked', 69),
 ('heard', 63),
 ('saw', 59),
 ('cried', 59),
 ('took', 56),
 ('turned', 55)]

In [23]:
multinerd_entities["EVE"]

['War',
 'World',
 'II',
 'Civil',
 'Cup',
 'American',
 'Revolution',
 'Festival',
 'League',
 'Film']

Сравневние: видим что тут тоже по-разному трактуется понятие event. В litbank это больше определяется по совершенному общему действию, а в multinerd это больше похоже на какое-то конкретное событие.

#### Кластеризация событий

**litbank**

In [24]:
#!pip install gensim==4.2.0

In [25]:
import gensim.downloader

w2v = gensim.downloader.load('word2vec-google-news-300')

In [26]:
import numpy as np

events = list(count_events(events_folder_path).keys())

embedded = []
presented_events = []
for event in events:
    if event in w2v:
        presented_events.append(event)
        embedded.append(w2v[event])
embedded = np.array(embedded)

In [27]:
from sklearn.cluster import KMeans

model = KMeans()
model.fit(embedded)
cluster_df = pd.DataFrame({"event" : presented_events, "cluster" : model.labels_})

In [28]:
for cluster in range(model.n_clusters):
    print(f"Cluster {cluster} sample:", *cluster_df[cluster_df['cluster'] == cluster].sample(10)['event'].values)

Cluster 0 sample: objects open fit Leaving proof saying advice stroke shift supposed
Cluster 1 sample: sorrow irritability struggles circumstance gravity seizure deaths elopement inference chatter
Cluster 2 sample: transported taken depicted added engaged regarded closed quoted became abated
Cluster 3 sample: complaining acknowledge amuse imagined hear stung persuasive amused rebuking appals
Cluster 4 sample: sparkled turned wound leapt flashed knelt walked harkened fallen nodded
Cluster 5 sample: barking frowning nodding Noticing clenching panting uttered sound sullen seated
Cluster 6 sample: raindrops smouldered dusted polluted deadened torn soft illumined dreary vapor
Cluster 7 sample: Locking overturning touching hopping mingling pointing serving flying organizing conjuring


На первый взгляд выделяющихся кластеров для litbank нет.

**multinerd**

In [29]:
events = df[df['Label'] == "EVE"]['Token'].values

embedded = []
presented_events = []
for event in events:
    if event in w2v:
        presented_events.append(event)
        embedded.append(w2v[event])
embedded = np.array(embedded)

In [30]:
model = KMeans()
model.fit(embedded)
cluster_df = pd.DataFrame({"event" : presented_events, "cluster" : model.labels_})

In [31]:
for cluster in range(model.n_clusters):
    print(f"Cluster {cluster} sample:", *cluster_df[cluster_df['cluster'] == cluster].sample(10)['event'].values)

Cluster 0 sample: Superleague CONCACAF Masters PSL Lottery ICC Medibank FIA Taekwondo Euroleague
Cluster 1 sample: Seven Nine Two Three Thirty Half Twenty Eighty One Six
Cluster 2 sample: Swiss Australian Danish Haitian Russian Iranian Afghan Liberian Canadian Congo
Cluster 3 sample: Series Shield Draft Best Doors Club Clockwork Slamdance Choice List
Cluster 4 sample: Kohima Ordina Rotterdam Pattaya Lübeck Luxembourg von Fes Zealand Asia
Cluster 5 sample: Rican Apertura Torneo Commune Telcel Los España Coppa Sud Antonio
Cluster 6 sample: Revolution Wars Patriotic Bloody Kapyong Servile Riot Guadalcanal Coalition Normandy
Cluster 7 sample: Midway Boer River Mercury A Mountain Desert June Springs Petitcodiac


Для multinerd кластеры выраженнее, в кластере 0 представлены спортивные дисциплины, в 4 - наименования различных лиг и федераций, в 5 - страны и города, в 6 - принадлежность к странам.

**Визуализация кластеров**

In [32]:
from sklearn.manifold import TSNE

events = list(count_events(events_folder_path).keys())

embedded = []
presented_events = []
for event in events:
    if event in w2v:
        presented_events.append(event)
        embedded.append(w2v[event])
embedded = np.array(embedded)

model = KMeans()
model.fit(embedded)
cluster_df = pd.DataFrame({"event" : presented_events, "cluster" : model.labels_})

tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(embedded)

In [33]:
colors = ['red', 'yellow', 'green', 'pink', 'black', 'gray', 'blue', 'brown']

def to_colors(label):
    return colors[label]

to_colors = np.vectorize(to_colors)

In [34]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="clusters")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=presented_events,
                                    color=to_colors(model.labels_)))

p.scatter(x="x1", y="x2", size=8, source=source, color='color')

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

Визуализация сильно зависит от параметров TSNE, поэтому объекты внутри кластеров могут находиться далеко друг от друга. Однако некоторые скопления можно углядеть.

In [35]:
events = df[df['Label'] == "EVE"]['Token'].values

embedded = []
presented_events = []
for event in events:
    if event in w2v:
        presented_events.append(event)
        embedded.append(w2v[event])
embedded = np.array(embedded)

model = KMeans()
model.fit(embedded)
cluster_df = pd.DataFrame({"event" : presented_events, "cluster" : model.labels_})

tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(embedded)

In [36]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=presented_events,
                                   color=to_colors(model.labels_)))

p.scatter(x="x1", y="x2", size=8, source=source, color='color')

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

Кластеры для визуализации multinerd более заметные.

В следующих частях домашнего задания вам понадобится train-test-dev разбиения. Авторы статей предлагают следующую структуру разбиения для корпуса LitBank: обучающее множество – 80 книг, валидационное – 10 книг, тестовое – 10 книг. Предложения из одного источника не должны попадать в разные сегменты разбиения.
Для корпуса MultiNERD_EN произведите стратифицированное разбиение по предложениям в соотношении 80%:10%:10%. Стратификацию производить в отношении пропорции по именованным сущностям (понятно, что с учётом всех вводных идеального разбиения не получится, но старайтесь сохранить распределения по типам сущностей для каждой и подвыборок).

In [37]:
from sklearn.model_selection import train_test_split

pathes = os.listdir('litbank_entities/tsv')
train_pathes, test_pathes = train_test_split(pathes, train_size=0.8, random_state=1)
val_pathes, test_pathes = train_test_split(test_pathes, train_size=0.5, random_state=1)
len(train_pathes), len(val_pathes), len(test_pathes)

(80, 10, 10)

## Часть 2. [3 балла] Извлечение именованных сущностей
Для каждой корпуса необходимо осуществить раздельно следующую процедуру:
1. [0.75 балла] Обучите стандартную модель для извлечения именованных сущностей, CNN-BiLSTM-CRF, для извлечения именованных *низкоуровневых именованных сущностей*, т.е. для самых коротких из вложенных сущностей. 
Модель устроена так: сверточная сеть на символах + эмбеддинги слов + двунаправленная LSTM сеть (модель последовательности) + CRF (глобальная нормализация).
2. [0.75 балла] Замените часть модели на символах и словах (CNN + эмбеддинги словах) на ELMo и / или BERT. Должна получиться модель ELMo / BERT + BiLSTM + CRF. 
3. [1.5 балла] Замените модель последовательности (BiLSTM) на другой слой, например, на Transformer. Должна получиться модель CNN  + Transformer + CRF.
В результате у вас будет 2 набора моделей - проверьте качество извлечения на тестовых данных.
- Примените модели первого корпуса ко всем (!!!) предложениям второго корпуса и наоборот.
- Составьте отчёт по качеству работы моделей в терминах извлечения по типам сущностей, которые встречаются в обоих корпусах. Метрику выберите самостоятельно.
- Проанализируйте ошибки моделей в терминах предсказания сущностям первого корпуса специфических типов для второго корпуса. И наоборот - когда специфическим типам второго корпуса модель, обученная на LitBank, прсваивает одну из своих категорий.

[бонус] Дообучите BERT для извлечения именованных сущностей.

[бонус] Используйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]

[бонус] Модифицируйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]: вместо эмбеддингов слов используйте ELMo и/или BERT. 

In [38]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [39]:
from torch.utils.data import Dataset, DataLoader

class BookDataset(Dataset):
    def __init__(self, pathes):
        super().__init__()
        self.data = []
        self.labels = []
        self.all_labels = set()
        self.read_books(pathes)
        self.len = len(self.data)
        
    def read_books(self, pathes):
        for path in pathes:
            folder_path = 'litbank_entities/tsv/'
            df = pd.read_csv(
                folder_path + path, sep='\t', quoting=csv.QUOTE_NONE,
                names=['word', 'label', '1', '2', '3', '4'], skip_blank_lines=False, index_col=False)
            blanks = np.append(-1, df[df['word'].isna()].index)
            for i in range(len(blanks) - 1):
                self.data.append(df[blanks[i] + 1:blanks[i + 1]]['word'].values)
                self.labels.append(df[blanks[i] + 1:blanks[i + 1]]['label'].values)
            self.all_labels |= set(df.dropna(subset='label')['label'].values)
        

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    

training_data = BookDataset(train_pathes)
val_data = BookDataset(val_pathes)
test_data = BookDataset(test_pathes)

In [156]:
def argmax(vec):
    _, idx = torch.max(vec, 1)
    return idx.item()

def prepare_sequence(seq, to_ix, padding=None):
    idxs = [to_ix[w] for w in seq]
    if padding:
        idxs += [to_ix['<PAD>'] for i in range(padding - len(idxs))]
    return torch.tensor(idxs, dtype=torch.long)

def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, char_vocab_size, tag_to_ix,
                 embedding_dim, hidden_dim, char_embedding_dim, dropout=0.5):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.char_embeds = nn.Embedding(char_vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim + char_embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        self.hidden = self.init_hidden()
        self.char_embedding_dim = char_embedding_dim
        self.cnn = nn.Conv2d(1, char_embedding_dim, kernel_size=(3, embedding_dim))
        self.cnn_fc = nn.Linear(char_embedding_dim * 1590, 100)
        

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        forward_var = init_alphas
        for feat in feats:
            alphas_t = []
            for next_tag in range(self.tagset_size):
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                trans_score = self.transitions[next_tag].view(1, -1)
                next_tag_var = forward_var + trans_score + emit_score
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    
    def _cnn(self, embedded):
        cnn_out = self.cnn(embedded)
        cnn_out = nn.functional.max_pool2d(cnn_out, kernel_size=(cnn_out.size(2), 1))
        #cnn_out = self.cnn_fc(cnn_out.view(-1))
        return cnn_out.view(-1, self.char_embedding_dim)

    def _get_lstm_features(self, sentence, chars):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence)
        char_features = []
        char_embeds = self.char_embeds(chars)
        cnn_res = self._cnn(char_embeds.unsqueeze(1))
        embeds = torch.cat([embeds, cnn_res], axis=1)
        embeds = embeds.view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []
            viterbivars_t = []

            for next_tag in range(self.tagset_size):
        
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, chars, tags):
        feats = self._get_lstm_features(sentence, chars)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence, chars):
        lstm_feats = self._get_lstm_features(sentence, chars)
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [157]:
word_to_ix = {}

def mapping(data, res, char_=False):
    for sentence, tags in data:
        for word in sentence:
            if char_:
                for char in word:
                    if char not in res:
                        res[char] = len(res)
            else:
                if word not in res:
                    res[word] = len(res)
                    
word_to_ix = {}
word_to_ix['<PAD>'] = 0
mapping(training_data, word_to_ix)
mapping(val_data, word_to_ix)
mapping(test_data, word_to_ix)

char_to_ix = {}
char_to_ix['<PAD>'] = 0
mapping(training_data, char_to_ix, True)
mapping(val_data, char_to_ix, True)
mapping(test_data, char_to_ix, True)

tag_to_ix = {list(training_data.all_labels)[i] : i for i in range(len(training_data.all_labels))}
tag_to_ix[START_TAG] = len(tag_to_ix)
tag_to_ix[STOP_TAG] = len(tag_to_ix)
tag_to_ix['<PAD>'] = tag_to_ix[STOP_TAG]

In [158]:
sentences = [' '.join(training_data[i][0]) for i in range(len(training_data))]
train_max = max(list(map(len, sentences)))
sentences = [' '.join(val_data[i][0]) for i in range(len(val_data))]
val_max = max(list(map(len, sentences)))
sentences = [' '.join(test_data[i][0]) for i in range(len(test_data))]
test_max = max(list(map(len, sentences)))
char_padding_size = max([train_max, val_max, test_max])

In [159]:
from tqdm.notebook import tqdm

START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 100
HIDDEN_DIM = 32
CHAR_EMBEDDING_DIM = 50

model = BiLSTM_CRF(len(word_to_ix), len(char_to_ix), tag_to_ix,
                   EMBEDDING_DIM, HIDDEN_DIM, CHAR_EMBEDDING_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    epoch_loss = 0
    for sentence, tags in tqdm(training_data):
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        chars_in = list(map(lambda x : prepare_sequence(x, char_to_ix, padding=char_padding_size), sentence))
        chars_in = torch.stack(chars_in)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        
        loss = model.neg_log_likelihood(sentence_in, chars_in, targets)
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()
    print(epoch_loss)

  0%|          | 0/6845 [00:00<?, ?it/s]

50952.90216374397


  0%|          | 0/6845 [00:00<?, ?it/s]

37092.343076229095


  0%|          | 0/6845 [00:00<?, ?it/s]

34033.68379163742


  0%|          | 0/6845 [00:00<?, ?it/s]

31838.3016204834


  0%|          | 0/6845 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [161]:
precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
chars_in = list(map(lambda x : prepare_sequence(x, char_to_ix, padding=char_padding_size), training_data[0][0]))
chars_in = torch.stack(chars_in)
pred = model(precheck_sent, chars_in)[1]

In [162]:
pred

[6, 6, 6, 6, 6, 6, 6, 6, 3, 9, 6]

In [164]:
ix_to_tag = {tag_to_ix[key] : key for key in tag_to_ix}

In [165]:
pred = list(map(lambda x : ix_to_tag[x], pred))

In [166]:
pred, training_data[0][1]

(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O'],
 array(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O'],
       dtype=object))

In [167]:
from sklearn.metrics import f1_score

f1_score(training_data[0][1], pred, average='macro')

1.0